<a href="https://colab.research.google.com/github/Cogitarian/MultCorp/blob/main/ELAN_PYMPI_LING.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

3.6.1 Instalacja i import
Pierwszy krok to instalacja biblioteki w wierszu poleceń:
pip install pympi-ling
Następnie wczytujemy bibliotekę pympi w Pythonie:

In [ ]:
import pympi

3.6.2 Wczytanie pliku .eaf
Przed każdą operacją należy wczytać plik .eaf, dlatego dla wygody określamy katalog roboczy:

In [ ]:
import os
eafpath = "plikiZelana"
os.chdir(eafpath)

Wczytujemy plik .eaf jako obiekt do klasy pympi.Eaf:

In [ ]:
eaffile = pympi.Eaf(eaf_file)

3.6.3 Przykładowe operacje w pympi-ling

Podaj listę warstw w pliku:

In [ ]:
listier = eaffile.get_tier_names()

Dodaj warstwę:

In [ ]:
eaffile.add_tier(tiername, ling= tiertype)

Zamiast tiername wpisujemy nazwę warstwy, a zamiast tiertype – nazwę typu dla warstwy. Typ musi być utworzony wcześniej w pliku .eaf.

Usuń warstwę:

In [ ]:
eaffile.remove_tier(oldname)

Zamiast oldname wpisujemy nazwę warstwy, którą chcemy usunąć.

Zmień nazwę warstwy:

In [ ]:
eaffile.rename_tier(oldname, newname)

Zamiast oldname wpisujemy nazwę warstwy, której nazwę zmieniamy, a zamiast newname – nową nazwę warstwy.

Połącz dwie warstwy w jedną:

In [ ]:
eaffile.merge_tiers(tier1,tier2, safe=True)

Zamiast tier1 i tier2 wpisujemy nazwy warstw, które chcemy połączyć.

Podaj listę anotacji na wybranej warstwie:

In [ ]:
annotations = eaffile.get_annotation_data_for_tier(tier)

Zamiast tier wpisujemy nazwę warstwy, z której chcemy uzyskać listę anotacji.

Usuń wszystkie anotacje na wybranej warstwie:

In [ ]:
eaffile.remove_all_annotations_from_tier(tier)

Zamiast tier wpisujemy nazwę warstwy, z której chcemy usunąć anotacje.

Dodaj anotację na wybranej warstwie:

In [ ]:
eaffile.add_annotation(tier,start,stop, value= text)

Zamiast tier wpisujemy nazwę warstwy, na której chcemy dodać anotacje złożone z początku – start, końca – start i etykiety – text.
3.6.4 Zachowywanie zmian w pliku .eaf

In [ ]:
eaffile.to_file(eaf_file)

3.6.4	Zachowywanie zmian w pliku .eaf
Zmiany zachowujemy, wywołując:


In [ ]:
	eaffile.to_file(eaf_file)

3.6.5	Wywołanie poleceń pympi w formie funkcji
Każdą z wyżej wymienionych funkcji możemy włączyć w nową, która wczyta plik .eaf i zwróci przerobiony plik, na przykład:

In [ ]:
def ADDtier(eaf_file, tiername, tiertype):
  eaffile = pympi.Eaf(eaf_file)
  eaffile.add_tier(tiername, ling= tiertype)
  eaffile.to_file(eaf_file)

3.6.6	Kopiowanie warstwy
Skopiuj warstwę w pliku .eaf, czyli zduplikuj warstwę. Do tej operacji potrzebna jest nowa funkcja:

In [ ]:
def DUPtier(eaf_file, oldname, newname, tiertype, parentier):
 eaffile = pympi.Eaf(eaf_file)
 annotations = eaffile.get_annotation_data_for_tier(oldname)
 eaffile.add_tier(newname, ling=tiertype, parent=parentier)
 for s in range(0,len(annotations)):
  (start, stop, text) = annotations
  eaffile.add_annotation(newname,start,stop,value=text)
 eaffile.to_file(eaf_file)
 del (eaffile,annotations)

Zamiast eaf_file wpisujemy nazwę pliku, w którym chcemy powtórzyć warstwę o nazwie oldname, a nowa warstwa będzie miała nazwę newname i typ tiertype oraz warstwę nadrzędną parentier.

Albo w innej wersji:

In [ ]:
def DUPtier2(eaf_file, oldname, newname):
 annofile = pympi.Elan.Eaf(eaf_file)
 tempfile = pympi.Elan.Eaf(pympi.__init__(" empty.eaf" ))
 annofile.copy_tier(tempfile,oldname)
 annofile.remove_tier(oldname)
 tempfile.rename_tier(oldname,newname)
 tempfile.copy_tier(annofile,newname)
 annofile.to_file(eaf_file)

3.6.7	Łączenie warstw anotacji
Połącz anotacje z dwóch plików w jeden, czyli funkcja dostępna w ELAN-ie w menu File > Merge transcriptions… Łączenie anotacji w jeden plik przydaje się, jeśli chcemy porównać anotacje dwóch anotatorów dla jednego nagrania. Pierwsza wersja tej funkcji dodaje warstwy anotacji z jednego pliku do drugiego i zachowuje jako nowy plik.

In [ ]:
def MEReafs(eaf_file1, eaf_file2):
  eaffile1 = pympi.Eaf(eaf_file1)
  eaffile2 = pympi.Eaf(eaf_file2)
  tierlist = eaffile1.get_tier_names()
  for tier in tierlist:
    annotations = eaffile1.get_annotation_data_for_tier(tier)
    eaffile2.add_tier(tier)
    for s in range(0,len(annotations)):
      (start, stop, text) = annotations
      eaffile2.add_annotation(tier,start,stop,value=text)
    eaffile2.to_file(eaf_file2)
  del(eaffile1,eaffile2,annotations)

Zamiast eaf_file1, eaf_file2 wpisujemy nazwy łączonych plików. Jednakże jeśli w dwóch plikach .eaf mamy takie same nazwy warstwy, to musimy je zmienić przed kopiowaniem do jednego pliku, bo ELAN nie pozwala na dwie warstwy o takich samych nazwach w jednym pliku. Dlatego druga funkcja uwzględnia zmiany nazwy warstw na nowy format złożony z nazwy warstwy i ostatnich trzech liter z nazwy pliku do oznaczenia anotatora. Załóżmy, że mamy dwa pliki anotacji dla tego samego nagrania wykonane przez różnych anotatorów: 81-60-S5-4-K-Mly.eaf i 81-60-S5-4-K-Occ.eaf, gdzie Mly i Occ oznaczają właśnie tych anotatorów. Łączymy oba pliki w nowy plik o nazwie 81-60-S5-4-K-Occ-Mly.eaf za pomocą funkcji:

In [ ]:
def MEReafs(eaf_file1, eaf_file2, new_file):
  import pympi
  import os
  eaffile1 = pympi.Eaf(eaf_file1)
  eaffile2 = pympi.Eaf(eaf_file2)
  eaffile_new = pympi.Eaf() # Create a new Eaf file
  # Extracting the last three characters of the file names without extension
  file1_name_suffix = os.path.splitext(os.path.basename(eaf_file1))
  file2_name_suffix = os.path.splitext(os.path.basename(eaf_file2))
  # Process the first file
  tierlist1 = eaffile1.get_tier_names()
  for tier in tierlist1:
    new_tier_name = f"{tier}_{file1_name_suffix}"
    annotations = eaffile1.get_annotation_data_for_tier(tier)
    eaffile_new.add_tier(new_tier_name)
    for start, stop, text in annotations:
      eaffile_new.add_annotation(new_tier_name, start, stop, value=text)
  # Process the second file
  tierlist2 = eaffile2.get_tier_names()
  for tier in tierlist2:
    new_tier_name = f"{tier}_{file2_name_suffix}"
    annotations = eaffile2.get_annotation_data_for_tier(tier)
    eaffile_new.add_tier(new_tier_name)
    for start, stop, text in annotations:
      eaffile_new.add_annotation(new_tier_name, start, stop, value=text)
  # Save the new file
  eaffile_new.to_file(new_file)
  del(eaffile1, eaffile2, eaffile_new)


Zamiast eaf_file1, eaf_file2 wpisujemy nazwy łączonych plików, a new_file to nazwa nowego pliku.

3.6.8	Zmiana typu warstwy
Zmień typ warstwy:

In [ ]:
def SETtype(eaf_file, tiername, tiertype):
  eaffile = pympi.Elan.Eaf(eaf_file)
  tierlist = eaffile.get_tier_names()
  if tiername in tierlist:
    annotations = eaffile.get_annotation_data_for_tier(tiername)
    eaffile.remove_tier(tiername)
    eaffile.add_tier(tiername, ling=tiertype)
    for s in range(0, len(annotations)):
      (start, stop, text)=annotations
      eaffile.add_annotation(tiername, start, stop, value=text)
    eaffile.to_file(eaf_file)
  del (eaffile, tiername, tiertype, tierlist)

Zamiast eaf_file wpisujemy nazwę pliku .eaf, a zamiast tiername – nazwę warstwy i zamiast tiertype – nazwę typu.
3.6.9	Zmiany anotacji za pomocą wyszukiwania i wymieniania tekstu lub tagu
Znajdź i zamień w anotacjach lub transkrypcjach na wybranej warstwie:

In [ ]:
def findANDrepl(eaf_file,tier,findtext,repltext):
  eaffile = pympi.Eaf(eaf_file)
  annotations = eaf_file.get_annotation_data_for_tier(tier)
  eaffile.remove_all_annotations_from_tier(tier)
  for s in range(0,len(annotations)):
    (start, stop, text) = annotations
    text = re.sub(findtext, repltext, text)
    eaffile.add_annotation(tier,start,stop, value=text)
  eaffile.to_file(eaf_file)


Zamiast eaf_file wpisujemy nazwę pliku .eaf, a zamiast tier – warstwy i findtext – szukany tekst, a repltext – tekst do zamiany w miejsce findtext. W miejscu re.sub można podstawić dowolne wyrażenie regularne, wedle którego chcemy dokonać korekty. W ten sposób możemy oczyścić transkrypcję z numerów, znaków niealfabetycznych albo zamienić wielkie litery na małe i tym podobne. Funkcja jest dostępna w ELAN-ie w menu Search > Find (and Replace).

1.1.1	Tokenizacja anotacji
Rozdziel anotacje lub transkrypcje na anotacje, gdzie spacja jest separatorem. Funkcja jest dostępna w ELAN-ie w menu Tier > Tokenize tier:


In [ ]:
def TOKtier(eaf_file, oldname, newname):
  transeaf = pympi.Eaf ( eaf_file )
  aucz = transeaf.get_annotation_data_for_tier(oldname)
  transeaf.add_tier(newname)
  for s in range ( 0 , len ( aucz ) ):
    (start , stop , text) = aucz
    wordList = re.sub ( "" , " " , text ).split ( )
    licz = len(wordList)
    if licz!=0:
      leng = stop-start
      word = leng/licz
      w = 0
      for wyr in wordList:
        starw = start+(word*w)
        starw = round ( starw )
        w = w + 1 # początek anotacji z 1 wyrazem
        stopw = start+(word*w) #koniec anotacji z 1 wyrazem
        stopw = round ( stopw )
        transeaf.add_annotation (newname , starw , stopw , value = wyr )
  transeaf.to_file(eaf_file)


Zamiast eaf_file wpisujemy nazwę pliku .eaf, a oldname i newname to nazwy warstw.

Bardziej złożone operacje pozwalają na uzupełnienie danych o uczestnikach lub anotatorach na podstawie pliku .csv albo na ustawienie skrótów i kolorów etykiet także z pliku .csv; wyznaczenie fragmentów anotacji, w których współwystępują określone słowa i gesty. Możliwe jest także zdefiniowanie zestawu reguł anotacji, wedle których chcieliśmy uporządkować anotacje w ramach wybranego systemu. Reguły dotyczyłyby usuwania, dodawania lub zmieniania anotacji na określonych warstwach zależnie od anotacji na innych warstwach. Nie dyskutujemy ich tutaj, gdyż wymagałoby to szczegółowego omówienia wybranego systemu anotacji, na przykład NEUROGES (Lausberg 2019).
